# RAG System Notebook

This notebook demonstrates the full Retrieval-Augmented Generation (RAG) pipeline.

## Purpose
This notebook integrates the retrieval and generation components to create a complete RAG system. It demonstrates:

1.  **Component Initialization**: Setting up the Retriever and Generator with the Knowledge Base.
2.  **Context Retrieval**: Querying the knowledge base to find relevant code examples and documentation.
3.  **Code Generation**: Using a local LLM (Ollama) augmented with retrieved context to generate Cirq code from natural language prompts.
4.  **RAG vs Direct LLM Comparison**: Comparing results with and without RAG context.

## Usage
Run this notebook to test the end-to-end RAG pipeline, from query to generated code.


In [1]:
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.cirq_rag_code_assistant.config import get_config, setup_logging
from src.rag.knowledge_base import KnowledgeBase
from src.rag.retriever import Retriever
from src.rag.generator import Generator

# Setup logging
setup_logging()

2025-12-06 00:04:39 | INFO     | src.cirq_rag_code_assistant.config.logging:setup_all:138 | Logging configuration completed


### Initialize Components
We need to:
1. Initialize the KnowledgeBase and load entries
2. Build or load the vector index
3. Create the Retriever and Generator

In [2]:
# Define paths
KNOWLEDGE_BASE_DIR = project_root / "data" / "knowledge_base"
VECTOR_INDEX_PATH = project_root / "data" / "models" / "vector_index"

# Initialize Knowledge Base and load entries
kb = KnowledgeBase(knowledge_base_path=KNOWLEDGE_BASE_DIR)
num_loaded = kb.load_from_directory()
print(f"Loaded {num_loaded} entries from knowledge base")

# Try to load existing index, or build new one
try:
    kb.load_index(VECTOR_INDEX_PATH)
    print(f"Loaded vector index: {kb.vector_store.size()} entries")
except FileNotFoundError:
    print("No existing index found, building new one...")
    kb.index_entries()
    kb.save_index(VECTOR_INDEX_PATH)
    print(f"Built and saved vector index: {kb.vector_store.size()} entries")

# Initialize Retriever with hybrid scoring
retriever = Retriever(knowledge_base=kb, use_hybrid_scoring=True)

# Initialize Generator (uses Ollama by default from config)
try:
    generator = Generator(retriever=retriever)
    print("\n✅ RAG components initialized successfully.")
except Exception as e:
    print(f"\n⚠️ Error initializing Generator: {e}")
    print("Make sure Ollama is running locally.")
    generator = None

2025-12-06 00:04:39 | INFO     | config.config_loader:load:93 | ✅ Loaded configuration from D:\University\Uni\Semester 7\Generative AI\Project\Cirq-RAG-Code-Assistant\config\config.dev.json
2025-12-06 00:04:39 | INFO     | src.rag.embeddings:__init__:97 | Loading embedding model: BAAI/bge-base-en-v1.5
2025-12-06 00:04:39 | INFO     | src.rag.embeddings:__init__:98 | Using device: cpu


2025-12-06 00:04:39,305 - sentence_transformers.SentenceTransformer - INFO - SentenceTransformer.py:227 - Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5


2025-12-06 00:04:43 | INFO     | src.rag.embeddings:__init__:106 | ✅ Embedding model loaded successfully
2025-12-06 00:04:43 | INFO     | src.rag.embeddings:__init__:113 | Embedding dimension: 768
2025-12-06 00:04:43 | INFO     | src.rag.vector_store:_init_faiss:139 | Initialized FAISS index
2025-12-06 00:04:43 | INFO     | src.rag.vector_store:__init__:120 | Initialized VectorStore with faiss backend
2025-12-06 00:04:43 | INFO     | src.rag.vector_store:__init__:121 | Embedding dimension: 768
2025-12-06 00:04:43 | INFO     | src.rag.knowledge_base:__init__:100 | Initialized KnowledgeBase with 0 entries
2025-12-06 00:04:43 | INFO     | src.rag.knowledge_base:load_from_jsonl:116 | Loading knowledge base from D:\University\Uni\Semester 7\Generative AI\Project\Cirq-RAG-Code-Assistant\data\knowledge_base\curated_designer_examples.jsonl
2025-12-06 00:04:43 | INFO     | src.rag.knowledge_base:load_from_jsonl:129 | Loaded 102 entries from D:\University\Uni\Semester 7\Generative AI\Project\Cir

Loaded 102 entries from knowledge base
Loaded vector index: 102 entries

✅ RAG components initialized successfully.


### Retrieval Demo
Let's see what the retriever finds for a sample query.

In [3]:
query = "Create a Bell state circuit"

results = retriever.retrieve(query, top_k=5)

print(f"Query: {query}")
print(f"Found {len(results)} results:")
for i, res in enumerate(results):
    entry = res['entry']
    score = res['score']
    boost = res.get('topic_boost', 0)
    print(f"\nResult {i+1} (Score: {score:.4f}, Boost: +{boost:.2f}):")
    print(f"  ID: {entry.get('id', 'N/A')}")
    print(f"  Topics: {entry.get('topics', [])}")
    print(f"  Task: {entry.get('task', 'N/A')[:100]}...")

Query: Create a Bell state circuit
Found 5 results:

Result 1 (Score: 0.9665, Boost: +0.30):
  ID: design_bell_state
  Topics: ['bell_state', 'entanglement', 'hadamard', 'cnot']
  Task: Design a Cirq circuit that prepares the Bell state (|00> + |11>)/sqrt(2) on two LineQubits and measu...

Result 2 (Score: 0.9596, Boost: +0.30):
  ID: design_bell_state_v2
  Topics: ['bell_state', 'entanglement', 'hadamard', 'cnot']
  Task: Create a Cirq function that returns a Bell-state preparation circuit on two given qubits, and then b...

Result 3 (Score: 0.9397, Boost: +0.30):
  ID: design_bell_state_v3
  Topics: ['bell_state', 'entanglement']
  Task: Design a Bell-state circuit that uses an explicit Moment structure instead of appending operations o...

Result 4 (Score: 0.8116, Boost: +0.15):
  ID: designer_teleportation_circuit
  Topics: ['teleportation', 'entanglement', 'bell_measurement']
  Task: Implement the Quantum Teleportation circuit to transfer a state from 'msg' to 'bob'....

Result 5 

---
## Generation Demo: RAG vs Direct LLM Comparison

This section compares two approaches:
1. **Direct LLM**: Sending only the user prompt to the LLM (no context)
2. **RAG Pipeline**: Retrieving relevant examples and including them as context

We'll show:
- The exact prompt sent to the LLM in each case
- The generated code from each approach

### Define the Test Query

In [4]:
# The test prompt for comparison
test_prompt = "Generate a quantum circuit that prepares a Bell state and measures it."
print(f"Test Prompt: {test_prompt}")

Test Prompt: Generate a quantum circuit that prepares a Bell state and measures it.


### 1. View the Direct LLM Prompt (No RAG)

This shows what gets sent to the LLM when we DON'T use RAG - just the raw prompt.

In [5]:
if generator is not None:
    # Build and display the direct prompt (no RAG context)
    direct_prompt = generator.build_direct_prompt(test_prompt)
    
    print("=" * 80)
    print("DIRECT LLM PROMPT (No RAG Context)")
    print("=" * 80)
    print(direct_prompt)
    print("=" * 80)
    print(f"\nPrompt length: {len(direct_prompt)} characters")
else:
    print("Generator not initialized.")

DIRECT LLM PROMPT (No RAG Context)
You are an expert quantum computing programmer specializing in Google's Cirq framework.

Your task is to generate syntactically correct, executable Cirq code based on the user's request.

User request: Generate a quantum circuit that prepares a Bell state and measures it.

Instructions:
1. Generate complete, executable Cirq code that fulfills the user's request
2. Follow Cirq best practices and conventions
3. Include necessary imports
4. Add comments explaining key steps
5. Ensure the code is syntactically correct and can be executed

Generated code:

Prompt length: 556 characters


### 2. View the RAG-Augmented Prompt (With Context)

This shows what gets sent to the LLM when we USE RAG - the prompt augmented with retrieved examples.

In [6]:
if generator is not None:
    # Build and display the RAG-augmented prompt
    rag_prompt_info = generator.build_prompt(test_prompt, top_k=3)
    
    print("=" * 80)
    print("RAG-AUGMENTED PROMPT (With Retrieved Context)")
    print("=" * 80)
    print(rag_prompt_info['full_prompt'])
    print("=" * 80)
    print(f"\nPrompt length: {len(rag_prompt_info['full_prompt'])} characters")
    print(f"Number of examples retrieved: {rag_prompt_info['num_examples']}")
else:
    print("Generator not initialized.")

RAG-AUGMENTED PROMPT (With Retrieved Context)
You are an expert quantum computing programmer specializing in Google's Cirq framework.

Your task is to generate syntactically correct, executable Cirq code based on the user's request and the provided examples.

Context from knowledge base:
Example 1:
Task: Design a Cirq circuit that prepares the Bell state (|00> + |11>)/sqrt(2) on two LineQubits and measures them in the computational basis.
Code:
import cirq

q0, q1 = cirq.LineQubit.range(2)

circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CNOT(q0, q1),
    cirq.measure(q0, q1, key='m'),
)



Example 2:
Task: Create a Cirq function that returns a Bell-state preparation circuit on two given qubits, and then build a main script that simulates and prints the measurement histogram.
Code:
import cirq


def bell_circuit(q0: cirq.Qid, q1: cirq.Qid) -> cirq.Circuit:
    circuit = cirq.Circuit(
        cirq.H(q0),
        cirq.CNOT(q0, q1),
        cirq.measure(q0, q1, key='m'),
    )
    return

### 3. View Just the Retrieved Context

This shows only the context that RAG attaches to the prompt.

In [7]:
if generator is not None:
    print("=" * 80)
    print("RETRIEVED CONTEXT (What RAG Adds to the Prompt)")
    print("=" * 80)
    print(rag_prompt_info['context'])
    print("=" * 80)
else:
    print("Generator not initialized.")

RETRIEVED CONTEXT (What RAG Adds to the Prompt)
Example 1:
Task: Design a Cirq circuit that prepares the Bell state (|00> + |11>)/sqrt(2) on two LineQubits and measures them in the computational basis.
Code:
import cirq

q0, q1 = cirq.LineQubit.range(2)

circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CNOT(q0, q1),
    cirq.measure(q0, q1, key='m'),
)



Example 2:
Task: Create a Cirq function that returns a Bell-state preparation circuit on two given qubits, and then build a main script that simulates and prints the measurement histogram.
Code:
import cirq


def bell_circuit(q0: cirq.Qid, q1: cirq.Qid) -> cirq.Circuit:
    circuit = cirq.Circuit(
        cirq.H(q0),
        cirq.CNOT(q0, q1),
        cirq.measure(q0, q1, key='m'),
    )
    return circuit


q0, q1 = cirq.LineQubit.range(2)

circuit = bell_circuit(q0, q1)

sim = cirq.Simulator()
result = sim.run(circuit, repetitions=1000)
print(result.histogram(key='m'))



Example 3:
Task: Design a Bell-state circuit that uses an exp

### 4. Generate Code: Direct LLM (No RAG)

Generate code using ONLY the LLM without any retrieved context.

In [8]:
if generator is not None:
    print("Generating code with Direct LLM (no RAG)...")
    print("This may take a minute...\n")
    
    try:
        direct_result = generator.generate_direct(test_prompt)
        
        print("=" * 80)
        print("DIRECT LLM GENERATED CODE (No RAG)")
        print("=" * 80)
        print(direct_result['code'])
        print("=" * 80)
        print(f"\nMetadata: {direct_result.get('metadata', {})}")
        print(f"Context used: {direct_result['context_used']} examples")
        
    except Exception as e:
        print(f"Direct generation failed: {e}")
        direct_result = None
else:
    print("Generator not initialized. Make sure Ollama is running.")
    direct_result = None

2025-12-06 00:04:43 | INFO     | src.rag.generator:generate_direct:437 | Generating code directly (no RAG) using ollama/qwen2.5-coder:14b-instruct-q4_K_M


Generating code with Direct LLM (no RAG)...
This may take a minute...



2025-12-06 00:05:52 | INFO     | src.rag.generator:generate_direct:496 | ✅ Direct code generation completed


DIRECT LLM GENERATED CODE (No RAG)
import cirq

# Define a quantum circuit with 2 qubits
q0, q1 = cirq.LineQubit.range(2)

# Create a circuit to prepare a Bell state |00> + |11>
bell_circuit = cirq.Circuit(
    # Apply a Hadamard gate to the first qubit to create superposition
    cirq.H(q0),
    # Apply a CNOT gate with q0 as control and q1 as target to entangle the qubits
    cirq.CNOT(q0, q1)
)

# Add measurement operations to both qubits
bell_circuit.append([cirq.measure(q0, key='q0'), cirq.measure(q1, key='q1')])

# Print the circuit
print("Bell State Circuit:")
print(bell_circuit)

# Simulate the circuit using a quantum simulator
simulator = cirq.Simulator()
result = simulator.run(bell_circuit, repetitions=10)

# Print the measurement results
print("\nMeasurement Results:")
print(result)

Metadata: {'code_length': 769, 'num_lines': 27, 'has_imports': True, 'has_cirq': True, 'imports': ['cirq'], 'detected_algorithm': 'teleportation'}
Context used: 0 examples


### 5. Generate Code: RAG Pipeline (With Context)

Generate code using the full RAG pipeline with retrieved examples as context.

In [9]:
if generator is not None:
    print("Generating code with RAG Pipeline...")
    print("This may take a minute...\n")
    
    try:
        rag_result = generator.generate(test_prompt)
        
        print("=" * 80)
        print("RAG PIPELINE GENERATED CODE (With Context)")
        print("=" * 80)
        print(rag_result['code'])
        print("=" * 80)
        print(f"\nMetadata: {rag_result.get('metadata', {})}")
        print(f"Context used: {rag_result['context_used']} examples")
        
    except Exception as e:
        print(f"RAG generation failed: {e}")
        rag_result = None
else:
    print("Generator not initialized. Make sure Ollama is running.")
    rag_result = None

2025-12-06 00:05:52 | INFO     | src.rag.generator:generate:192 | Retrieving context for query: Generate a quantum circuit that prepares a Bell st...
2025-12-06 00:05:52 | INFO     | src.rag.generator:generate:213 | Generating code using ollama/qwen2.5-coder:14b-instruct-q4_K_M


Generating code with RAG Pipeline...
This may take a minute...



2025-12-06 00:07:03 | INFO     | src.rag.generator:generate:272 | ✅ Code generation completed


RAG PIPELINE GENERATED CODE (With Context)
import cirq

def bell_state_circuit(q0: cirq.Qid, q1: cirq.Qid) -> cirq.Circuit:
    """
    Prepares a Bell state (|00> + |11>)/sqrt(2) on two qubits and measures them.
    
    Args:
        q0: The first qubit.
        q1: The second qubit.
        
    Returns:
        A Cirq circuit that prepares the Bell state and measures it.
    """
    # Create a quantum circuit
    circuit = cirq.Circuit(
        # Apply Hadamard gate to the first qubit to create superposition |0> + |1>
        cirq.H(q0),
        
        # Apply CNOT gate with q0 as control and q1 as target to entangle the qubits
        cirq.CNOT(q0, q1),
        
        # Measure both qubits in the computational basis
        cirq.measure(q0, q1, key='m')
    )
    
    return circuit

# Define two qubits
q0, q1 = cirq.LineQubit.range(2)

# Generate the Bell state circuit
circuit = bell_state_circuit(q0, q1)

# Print the circuit to visualize it
print("Circuit:")
print(circuit)



### 6. Side-by-Side Comparison Summary

Compare key metrics between the two approaches.

In [10]:
if direct_result is not None and rag_result is not None:
    print("=" * 80)
    print("COMPARISON SUMMARY: Direct LLM vs RAG Pipeline")
    print("=" * 80)
    
    direct_meta = direct_result.get('metadata', {})
    rag_meta = rag_result.get('metadata', {})
    
    print(f"\n{'Metric':<25} {'Direct LLM':<20} {'RAG Pipeline':<20}")
    print("-" * 65)
    print(f"{'Code Length (chars)':<25} {direct_meta.get('code_length', 'N/A'):<20} {rag_meta.get('code_length', 'N/A'):<20}")
    print(f"{'Number of Lines':<25} {direct_meta.get('num_lines', 'N/A'):<20} {rag_meta.get('num_lines', 'N/A'):<20}")
    print(f"{'Has Cirq Import':<25} {direct_meta.get('has_cirq', 'N/A'):<20} {rag_meta.get('has_cirq', 'N/A'):<20}")
    print(f"{'Detected Algorithm':<25} {direct_meta.get('detected_algorithm', 'N/A'):<20} {rag_meta.get('detected_algorithm', 'N/A'):<20}")
    print(f"{'Context Examples Used':<25} {direct_result['context_used']:<20} {rag_result['context_used']:<20}")
    
    print("\n" + "=" * 80)
    print("KEY INSIGHT:")
    print("The RAG pipeline provides relevant code examples to the LLM, which can")
    print("improve generated code quality by following established patterns and best practices.")
    print("=" * 80)
else:
    print("Cannot compare - one or both generations failed.")

COMPARISON SUMMARY: Direct LLM vs RAG Pipeline

Metric                    Direct LLM           RAG Pipeline        
-----------------------------------------------------------------
Code Length (chars)       769                  1196                
Number of Lines           27                   44                  
Has Cirq Import           1                    1                   
Detected Algorithm        teleportation        teleportation       
Context Examples Used     0                    3                   

KEY INSIGHT:
The RAG pipeline provides relevant code examples to the LLM, which can
improve generated code quality by following established patterns and best practices.
